In [6]:
import pandas as pd
import matplotlib.pyplot as plt 

from bs4 import BeautifulSoup
import requests
import time
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [7]:
df_movies = pd.read_csv('movies.csv') 

In [8]:
df_ratings = pd.read_csv('ratings.csv')

In [9]:
df_links = pd.read_csv('links.csv', dtype = object)
df_links.fillna(value=0, inplace=True)

In [10]:
df_tags = pd.read_csv('tags.csv')

In [ ]:
df_movies

In [ ]:
df_ratings

In [ ]:
df_links

In [ ]:
df_tags

In [11]:
header = {"accept-language": "es-ES"}

In [ ]:
def sacar_sinopsis(df_links, header):
    results = []
    for i, tmdbId in enumerate(df_links["tmdbId"]):
        link = "https://www.themoviedb.org/movie/" + str(tmdbId)
        r = requests.get(link, headers = header)
        bs = BeautifulSoup(r.content, "html.parser")
        try:
            sinopsis = bs.p.get_text()
            movie_id = df_links.at[i,"movieId"]
            results.append([movie_id,tmdbId,sinopsis])
        except:
            results.append([tmdbId,"No se encontro sinopsis"])
            continue
        time.sleep(1)
        print(f"{i+1}/{len(df_links)} completed")

    df_sinopsis = pd.DataFrame(results,columns=['movieId','tmdbId','sinopsis'])
    df_sinopsis.to_csv('sinopsis.csv', index=False)
    return df_sinopsis
df_sinopsis = sacar_sinopsis(df_links, header)

In [12]:
df_sinopsis = pd.read_csv('sinopsis.csv')

In [13]:
df_usuario = pd.read_csv('Usuario_0.csv')

In [14]:
def juntarMoviesSinopsis(df_movies, df_sinopsis, movieId, movies_sinopsis):
    # Unir los dataframes en función de la columna 'movieId'
    df_movies_sinopsis = pd.merge(df_movies, df_sinopsis, on='movieId')

    # Eliminar columna tmdbId
    df_movies_sinopsis = df_movies_sinopsis.drop('tmdbId', axis=1)
    
    # Guardar el resultado en un nuevo archivo CSV
    df_movies_sinopsis.to_csv('movies_sinopsis.csv', index=False)    
    return df_movies_sinopsis

In [15]:
#Comprobar que crea y elimina y guarda en un fichero csv
df_movies_sinopsis = juntarMoviesSinopsis(df_movies, df_sinopsis, 'movieId', 'movies_sinopsis.csv')
print(df_movies_sinopsis)

      movieId                                      title  \
0           1                           Toy Story (1995)   
1           2                             Jumanji (1995)   
2           3                    Grumpier Old Men (1995)   
3           4                   Waiting to Exhale (1995)   
4           5         Father of the Bride Part II (1995)   
...       ...                                        ...   
9737   193581  Black Butler: Book of the Atlantic (2017)   
9738   193583               No Game No Life: Zero (2017)   
9739   193585                               Flint (2017)   
9740   193587        Bungo Stray Dogs: Dead Apple (2018)   
9741   193609        Andrew Dice Clay: Dice Rules (1991)   

                                           genres  \
0     Adventure|Animation|Children|Comedy|Fantasy   
1                      Adventure|Children|Fantasy   
2                                  Comedy|Romance   
3                            Comedy|Drama|Romance   
4             

In [16]:
df_movies_sinopsis

,movieId,title,genres,sinopsis
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"Los juguetes de Andy, un niño de seis años, te..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,Alan Parris queda atrapado en un juego de mesa...
2,3,Grumpier Old Men (1995),Comedy|Romance,El verano ha llegado a Minnesotta y la relació...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"Basada en el best seller de Terry McMillan, la..."
4,5,Father of the Bride Part II (1995),Comedy,"Tras la boda de su querida hija Anne, George B..."
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,Susurros se convierten en titulares gritando e...
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,Es una precuela situada 6000 años antes de la ...
9739,193585,Flint (2017),Drama,Una mujer se ocupa del escándalo del agua tóxi...
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,La historia está protagonizada por una liga de...


In [17]:
def get_sinopsis1(title):
    # Leer el archivo CSV
    df_movies_sinopsis = pd.read_csv("movies_sinopsis.csv")

    # Buscar la sinopsis en el archivo CSV
    for i, row in df_movies_sinopsis.iterrows():
        if row["title"] == title:
            return row["sinopsis"]
    return "Sinopsis no encontrada"

# Ejemplo de uso
title = "Leaving Las Vegas (1995)"
sinopsisT = get_sinopsis1(title)
print(sinopsisT)

Ben es un alcohólico cuya vida se está yendo al traste. Decide irse a Las Vegas, la ciudad donde los bares nunca cierran, para beber hasta morir. Allí conoce a Sera, una prostituta. Se aceptan, se enamoran, se separan y se reencuentran. Una maravillosa historia sobre dos personas que aprenden a aceptarse el uno al otro, sobre cómo en medio de la oscuridad puede surgir una pequeña llama de esperanza en sus vidas.


In [18]:
def get_movie_info(buscar):
    # Leer el archivo CSV
    df = pd.read_csv("movies_sinopsis.csv")
    search_by = {"id": "movieId", "titulo": "title"}
    
    # Buscar la pelicula en el archivo CSV
    for i, row in df.iterrows():
        if row[search_by[buscar[0]]] == buscar[1]:
            return row[["movieId", "title", "genres","sinopsis"]]
    return "Pelicula no encontrada"

# Ejemplo de uso mediante id y titulo
buscar = ("id", 1)
buscar = ("titulo", "Toy Story (1995)")
movie_info = get_movie_info(buscar)
print(movie_info)

movieId                                                     1
title                                        Toy Story (1995)
genres            Adventure|Animation|Children|Comedy|Fantasy
sinopsis    Los juguetes de Andy, un niño de seis años, te...
Name: 0, dtype: object


In [ ]:
# class procesarTexto():
    
#     def tokenize(text):
        
#         return nltk.word_tokenize(text)
    
#     def clean_text(text):
#         # Eliminar números
#         text = re.sub(r'\d+', '', text)
#         # Eliminar signos de puntuación
#         text = re.sub(r'[^\w\s]', '', text)
#         # Convertir a minúsculas
#         text = text.lower()
#         return text

#     def remove_stopwords(text):
#         # Cargar lista de stopwords en español
#         stop_words = set(stopwords.words("spanish"))
#         # Tokenizar el texto
#         tokens = word_tokenize(text)
#         # Eliminar stopwords
#         filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
#         # Unir los tokens en una cadena de texto
#         filtered_text = ' '.join(filtered_tokens)
#         return filtered_text
    
#     def stem_text(text):
#         # Tokenizar el texto
#         tokens = word_tokenize(text)
#         # Crear un objeto stemmer
#         stemmer = SnowballStemmer("spanish")
#         # Aplicar stemming a cada token
#         stemmed_tokens = [stemmer.stem(token) for token in tokens]
#         # Unir los tokens en una cadena de texto
#         stemmed_text = ' '.join(stemmed_tokens)
#         return stemmed_text
    
#     def word_frequency(text):
#         # Tokenizar el texto
#         tokens = word_tokenize(text)
#         # Contar la frecuencia de cada token
#         freq = nltk.FreqDist(tokens)
#         # Devolver diccionario con la frecuencia de cada palabra
#         return dict(freq)
    
# def get_sinopsis(title):
# # Leer el archivo CSV
#     df = pd.read_csv("movies_sinopsis.csv")

#     # Buscar la sinopsis en el archivo CSV
#     for i, row in df.iterrows():
#         if row["title"] == title:
#             sinopsis = row["sinopsis"]
#             sinopsis = procesarTexto.clean_text(sinopsis)
#             sinopsis = procesarTexto.remove_stopwords(sinopsis)
#             sinopsis = procesarTexto.stem_text(sinopsis)
#             frequency = procesarTexto.word_frequency(sinopsis)
#             return sinopsis, frequency
#     return "Sinopsis no encontrada"

# # Ejemplo de uso
# title = "Toy Story (1995)"
# sinopsis, frequency = get_sinopsis(title)
# print(sinopsis)
# print(frequency) 

In [ ]:
# def process_sinopsis(search_param):
#     movie_info = get_movie_info(search_param)
#     sinopsis = movie_info["sinopsis"]
#     sinopsis = clean_text(sinopsis)
#     sinopsis = remove_stopwords(sinopsis)
#     sinopsis = stem_text(sinopsis)
#     frequency = word_frequency(sinopsis)
#     return sinopsis, frequency

# search_param = ("id", 1)
# sinopsis, frequency = process_sinopsis(search_param)
# print(sinopsis)
# print(frequency)